<a href="https://colab.research.google.com/github/MUbarak123-56/qiskit-work/blob/master/optimization/converters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q qiskit
!pip install -q pylatexenc
#!pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src
!pip install qiskit-aer
!pip install qiskit-machine-learning
!pip install qiskit-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw

In [2]:
from qiskit_optimization import QuadraticProgram

In [3]:
qp = QuadraticProgram()
qp.binary_var("x")
qp.binary_var("y")
qp.integer_var(lowerbound=0, upperbound=7, name="z")

qp.maximize(linear={"x": 2, "y": 1, "z": 1})
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="LE", rhs=5.5, name="xyz_leq")
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="GE", rhs=2.5, name="xyz_geq")
print(qp.prettyprint())

Problem name: 

Maximize
  2*x + y + z

Subject to
  Linear constraints (2)
    x + y + z <= 5.5  'xyz_leq'
    x + y + z >= 2.5  'xyz_geq'

  Integer variables (1)
    0 <= z <= 7

  Binary variables (2)
    x y



### Inequality to Equality

In [4]:
from qiskit_optimization.converters import InequalityToEquality

In [5]:
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp)
print(qp_eq.prettyprint())

Problem name: 

Maximize
  2*x + y + z

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack + y + z == 5  'xyz_leq'
    x - xyz_geq@int_slack + y + z == 3  'xyz_geq'

  Integer variables (3)
    0 <= z <= 7
    0 <= xyz_leq@int_slack <= 5
    0 <= xyz_geq@int_slack <= 6

  Binary variables (2)
    x y



### Integer to Binary

In [8]:
from qiskit_optimization.converters import IntegerToBinary

In [9]:
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.prettyprint())

Problem name: 

Maximize
  2*x + y + z@0 + 2*z@1 + 4*z@2

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack@0 + 2*xyz_leq@int_slack@1 + 2*xyz_leq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 5  'xyz_leq'
    x - xyz_geq@int_slack@0 - 2*xyz_geq@int_slack@1 - 3*xyz_geq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 3  'xyz_geq'

  Binary variables (11)
    x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
    xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2



### Linear Equality to Penalty (Quadratic Unconstrained Binary Optimization)

In [10]:
from qiskit_optimization.converters import LinearEqualityToPenalty

In [11]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.prettyprint())

Problem name: 

Maximize
  -22*x^2 + 22*x*xyz_geq@int_slack@0 + 44*x*xyz_geq@int_slack@1
  + 66*x*xyz_geq@int_slack@2 - 22*x*xyz_leq@int_slack@0
  - 44*x*xyz_leq@int_slack@1 - 44*x*xyz_leq@int_slack@2 - 44*x*y - 44*x*z@0
  - 88*x*z@1 - 176*x*z@2 - 11*xyz_geq@int_slack@0^2
  - 44*xyz_geq@int_slack@0*xyz_geq@int_slack@1
  - 66*xyz_geq@int_slack@0*xyz_geq@int_slack@2 - 44*xyz_geq@int_slack@1^2
  - 132*xyz_geq@int_slack@1*xyz_geq@int_slack@2 - 99*xyz_geq@int_slack@2^2
  - 11*xyz_leq@int_slack@0^2 - 44*xyz_leq@int_slack@0*xyz_leq@int_slack@1
  - 44*xyz_leq@int_slack@0*xyz_leq@int_slack@2 - 44*xyz_leq@int_slack@1^2
  - 88*xyz_leq@int_slack@1*xyz_leq@int_slack@2 - 44*xyz_leq@int_slack@2^2
  + 22*y*xyz_geq@int_slack@0 + 44*y*xyz_geq@int_slack@1
  + 66*y*xyz_geq@int_slack@2 - 22*y*xyz_leq@int_slack@0
  - 44*y*xyz_leq@int_slack@1 - 44*y*xyz_leq@int_slack@2 - 22*y^2 - 44*y*z@0
  - 88*y*z@1 - 176*y*z@2 + 22*z@0*xyz_geq@int_slack@0
  + 44*z@0*xyz_geq@int_slack@1 + 66*z@0*xyz_geq@int_slack@2
  - 22*